**Model Prediction Script**

In [ ]:
# ----------------------------------------------------------------
# PRE-PROCESSING CODES USED FOR DATASETS BEING USED FOR PREDICTION
# ----------------------------------------------------------------

# Steps to Extract Player vs. Venue Data
# 1. Load the Datasets
# import pandas as pd
# # Load deliveries and matches data
# deliveries_df = pd.read_csv("deliveries.csv")
# matches_df = pd.read_csv("matches.csv")
# # Rename 'id' column in matches_df to 'match_id' for merging
# matches_df.rename(columns={'id': 'match_id'}, inplace=True)
# 2. Merge Deliveries with Matches to Get Venue Data
# # Select relevant columns from matches_df
# matches_subset = matches_df[['match_id', 'venue']]
# # Merge deliveries with matches to include venue information
# merged_df = deliveries_df.merge(matches_subset, on='match_id', how='left')
# # Check if the venue column is successfully added
# print(merged_df.head())
# 3. Analyze Player Performance by Venue
# For Batsmen (Total Runs & Strike Rate at Each Venue)
# batsman_stats = merged_df.groupby(['batter', 'venue']).agg(
# total_runs=('batsman_runs', 'sum'),
# balls_faced=('ball', 'count')
# ).reset_index()
# # Calculate strike rate
# batsman_stats['strike_rate'] = (batsman_stats['total_runs'] / batsman_stats['balls_faced']) * 100
# print(batsman_stats.head())  # Display results
# For Bowlers (Wickets & Economy Rate at Each Venue)
# bowler_stats = merged_df.groupby(['bowler', 'venue']).agg(
# wickets_taken=('is_wicket', 'sum'),
# balls_bowled=('ball', 'count')
# ).reset_index()
# # Calculate economy rate
# runs_conceded = merged_df.groupby(['bowler',
# 'venue'])['total_runs'].sum().reset_index(name='runs_conceded')
# bowler_stats = bowler_stats.merge(runs_conceded, on=['bowler', 'venue'], how='left')
# bowler_stats['economy_rate'] = (bowler_stats['runs_conceded'] / bowler_stats['balls_bowled']) *
# 6
# print(bowler_stats.head())  # Display results
# 4. Save Processed Data
# batsman_stats.to_csv("player_vs_venue_batting.csv", index=False)
# bowler_stats.to_csv("player_vs_venue_bowling.csv", index=False)
# Steps to Extract Phase-wise Performance at Each Venue
# import pandas as pd
# # Load merged dataset
# df = pd.read_csv("merged_deliveries_matches.csv")
# # Define match phases
# def categorize_phase(over):
# if over <= 6:
# return "Powerplay"
# elif over <= 15:
# return "Middle Overs"
# else:
# return "Death Overs"
# # Apply phase categorization
# df['phase'] = df['over'].apply(categorize_phase)
# # Check data
# print(df[['batter', 'venue', 'over', 'phase']].head())
# Compute Batting Performance by Phase & Venue
# # Aggregate batting stats based on player, venue, and phase
# batsman_phase_stats = df.groupby(['batter', 'venue', 'phase']).agg(
# total_runs=('batsman_runs', 'sum'),
# balls_faced=('ball', 'count'),
# sixes=('batsman_runs', lambda x: (x == 6).sum()),
# fours=('batsman_runs', lambda x: (x == 4).sum())
# ).reset_index()
# # Calculate strike rate
# batsman_phase_stats['strike_rate'] = (batsman_phase_stats['total_runs'] /
# batsman_phase_stats['balls_faced']) * 100
# # Save results
# batsman_phase_stats.to_csv("player_phasewise_venue_batting.csv", index=False)
# print(" Batting phase-wise stats saved as 'player_phasewise_venue_batting.csv'")
# print(batsman_phase_stats.head())
# Compute Bowling Performance by Phase & Venue
# # Aggregate bowling stats based on bowler, venue, and phase
# bowler_phase_stats = df.groupby(['bowler', 'venue', 'phase']).agg(
# wickets_taken=('is_wicket', 'sum'),
# balls_bowled=('ball', 'count'),
# runs_conceded=('total_runs', 'sum')
# ).reset_index()
# # Calculate economy rate & bowling strike rate
# bowler_phase_stats['economy_rate'] = (bowler_phase_stats['runs_conceded'] /
# bowler_phase_stats['balls_bowled']) * 6
# bowler_phase_stats['bowling_strike_rate'] = bowler_phase_stats['balls_bowled'] /
# bowler_phase_stats['wickets_taken']
# # Save results
# bowler_phase_stats.to_csv("player_phasewise_venue_bowling.csv", index=False)
# print(" Bowling phase-wise stats saved as 'player_phasewise_venue_bowling.csv'")
# print(bowler_phase_stats.head())
# Steps to Extract Player vs. Player Data
# Aggregate Batting Stats Against Each Bowler
# import pandas as pd
# Load merged dataset
# df = pd.read_csv("merged_deliveries_matches.csv")
# Group by batter and bowler to calculate runs, balls faced, and dismissals
# pvp_stats = df.groupby(['batter', 'bowler']).agg(
# total_runs=('batsman_runs', 'sum'),
# balls_faced=('ball', 'count'),
# dismissals=('is_wicket', 'sum'),
# sixes=('batsman_runs', lambda x: (x == 6).sum()),
# fours=('batsman_runs', lambda x: (x == 4).sum())
# ).reset_index()
# # Calculate strike rate and average (handling zero dismissals)
# pvp_stats['strike_rate'] = (pvp_stats['total_runs'] / pvp_stats['balls_faced']) * 100
# pvp_stats['average'] = pvp_stats['total_runs'] / pvp_stats['dismissals'].replace(0, 1)  # Avoid
# division by zero
# # Save the results
# pvp_stats.to_csv("player_vs_player.csv", index=False)
# print(" Player vs. Player stats saved as 'player_vs_player.csv'.")
# print(pvp_stats.head())
# Calculating expected and other imp columns for batter data
# import pandas as pd
# import numpy as np
# # Load your dataset
# df = pd.read_excel("batting_with_fantasy_points_phase.xlsx")
# # Step 1: Basic Derived Stats
# df['strike_rate'] = (df['total_runs'] / df['balls_faced']) * 100
# df['Boundary_Percentage'] = ((df['sixes'] + df['fours']) / df['balls_faced']) * 100
# df['PPS_Batting'] = df['total_runs'] + df['fours'] * 1 + df['sixes'] * 2  # Based on fantasy rules
# df['Matches_at_Venue']
# matches_df = df.groupby(['batter',
# 'venue'])['match_id'].nunique().reset_index(name='Matches_at_Venue')
# df = df.merge(matches_df, on=['batter', 'venue'], how='left')
# # Step 3: Average Runs & VPS (Venue Points Score)
# venue_stats = df.groupby(['batter', 'venue']).agg({
# 'total_runs': 'sum',
# 'balls_faced': 'sum',
# 'sixes': 'sum',
# 'fours': 'sum',
# 'PPS_Batting': 'sum',
# 'Matches_at_Venue': 'first'  # Already cleaned earlier
# }).reset_index()
# # Calculate averages
# venue_stats['Avg_Runs_at_Venue'] = venue_stats['total_runs'] / venue_stats['Matches_at_Venue']
# venue_stats['VPS_Batting'] = venue_stats['PPS_Batting'] / venue_stats['Matches_at_Venue']
# # Expected Stats
# venue_stats['Expected_Runs_per_Match'] = venue_stats['total_runs'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Balls_Faced_per_Match'] = venue_stats['balls_faced'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Sixes_per_Match'] = venue_stats['sixes'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Fours_per_Match'] = venue_stats['fours'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Strike_Rate_per_Match'] = (venue_stats['Expected_Runs_per_Match'] /
# venue_stats['Expected_Balls_Faced_per_Match']) * 100
# venue_stats['Expected_Boundary_Percentage_per_Match'] =
# ((venue_stats['Expected_Sixes_per_Match'] + venue_stats['Expected_Fours_per_Match']) /
# venue_stats['Expected_Balls_Faced_per_Match']) * 100
# # Step 4: Fantasy Bonuses per phase-row
# def bonus_runs(row):
# bonus = 0
# if row['total_runs'] >= 100:
# bonus = 16
# elif row['total_runs'] >= 75:
# bonus = 12
# elif row['total_runs'] >= 50:
# bonus = 8
# elif row['total_runs'] >= 25:
# bonus = 4
# return bonus
# def duck_penalty(row):
# return -2 if row['total_runs'] == 0 and row['balls_faced'] > 0 else 0
# def sr_bonus(row):
# sr = row['strike_rate']
# if sr >= 100:
# return 8
# elif sr >= 80:
# return 6
# elif sr >= 70:
# return 4
# elif sr >= 60:
# return 2
# else:
# return 0
# df['25R_Bonus'] = df['total_runs'].apply(lambda x: 4 if x >= 25 else 0)
# df['50R_Bonus'] = df['total_runs'].apply(lambda x: 8 if x >= 50 else 0)
# df['75R_Bonus'] = df['total_runs'].apply(lambda x: 12 if x >= 75 else 0)
# df['100R_Bonus'] = df['total_runs'].apply(lambda x: 16 if x >= 100 else 0)
# df['Duck_Penalty'] = df.apply(duck_penalty, axis=1)
# df['SR'] = df['strike_rate']
# df['SR_Bonus'] = df.apply(sr_bonus, axis=1)
# #FPbatting =(Runs×1)+(Boundaries×4)+(Sixes×6)+(25R_Bonus)+(50R_Bonus)+(75R_Bonus)+(
#  100R_Bonus)+(SR_Bonus)+(Duck_Penalty)
# # Step 5: Merge Venue-Level Stats into Phase-Level Data
# df = df.merge(
# venue_stats[
# ['batter', 'venue',
# 'total_runs', 'balls_faced', 'sixes', 'fours',
# 'Expected_Runs_per_Match', 'Expected_Balls_Faced_per_Match',
# 'Expected_Sixes_per_Match', 'Expected_Fours_per_Match',
# 'Expected_Strike_Rate_per_Match', 'Expected_Boundary_Percentage_per_Match']
# ].rename(columns={
# 'total_runs': 'total_runs_at_venue',
# 'balls_faced': 'total_balls_faced_at_venue',
# 'sixes': 'total_sixes_at_venue',
# 'fours': 'total_fours_at_venue'
# }),
# on=['batter', 'venue'],
# how='left'
# )
# # Step 6: Export final result
# df.to_excel("batting_final_stats.xlsx", index=False)
# print(“ Batting fantasy stats generated successfully!")
# Calculating expected stats and other imp columns in bowling stats
# import pandas as pd
# import numpy as np
# # Load bowling phase-venue data
# df = pd.read_excel("bowling_phase_venue.xlsx")  # Replace with actual filename
# # Step 1: Basic Calculated Columns
# df['economy_rate'] = df['runs_conceded'] / (df['balls_bowled'] / 6)
# df['bowling_strike_rate'] = df['balls_bowled'] / df['wickets_taken'].replace(0, np.nan)  # Avoid
# div by 0
# df['PPS_Bowling'] = df['wickets_taken'] * 25  # Assuming 25 points per wicket
# # Step 2: Matches_at_Venue (assumed cleaned/fixed separately)
# # If you don’t have match IDs, you're likely using cleaned values.
# # Make sure not to double-count per phase.
# # Step 3: Venue-Level Aggregation
# venue_stats = df.groupby(['Player_Name', 'venue']).agg({
# 'wickets_taken': 'sum',
# 'balls_bowled': 'sum',
# 'runs_conceded': 'sum',
# 'PPS_Bowling': 'sum',
# 'Matches_at_Venue': 'first'  # Already adjusted earlier
# }).reset_index()
# # Averages at Venue
# venue_stats['Avg_Wickets_at_Venue'] = venue_stats['wickets_taken'] /
# venue_stats['Matches_at_Venue']
# venue_stats['VPS_Bowling'] = venue_stats['PPS_Bowling'] / venue_stats['Matches_at_Venue']
# # Expected Stats
# venue_stats['Expected_Wickets_per_Match'] = venue_stats['wickets_taken'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Balls_Bowled_per_Match'] = venue_stats['balls_bowled'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Runs_Conceded_per_Match'] = venue_stats['runs_conceded'] /
# venue_stats['Matches_at_Venue']
# venue_stats['Expected_Economy_Rate_per_Match'] =
# venue_stats['Expected_Runs_Conceded_per_Match'] /
# (venue_stats['Expected_Balls_Bowled_per_Match'] / 6)
# venue_stats['Expected_Bowling_Strike_Rate_per_Match'] =
# venue_stats['Expected_Balls_Bowled_per_Match'] /
# venue_stats['Expected_Wickets_per_Match'].replace(0, np.nan)
# # Step 4: Bonus Calculations (Per Row)
# def three_wicket_bonus(x):
# return 4 if x >= 3 else 0
# def four_wicket_bonus(x):
# return 8 if x >= 4 else 0
# def five_wicket_bonus(x):
# return 16 if x >= 5 else 0
# def economy_bonus(er):
# if er < 4:
# return 6
# elif er < 5:
# return 4
# elif er < 6:
# return 2
# else:
# return 0
# df['3W_Bonus'] = df['wickets_taken'].apply(three_wicket_bonus)
# df['4W_Bonus'] = df['wickets_taken'].apply(four_wicket_bonus)
# df['5W_Bonus'] = df['wickets_taken'].apply(five_wicket_bonus)
# df['Maiden_Bonus'] = 0  # Optional: update if maiden overs available
# df['ER'] = df['economy_rate']
# df['Economy_Bonus'] = df['ER'].apply(economy_bonus)
# # Total Fantasy Points (FP) for Bowling
# #FPbowling =(Wickets×25)+(LBW_Bowled×8)+(3W_Bonus)+(4W_Bonus)+(5W_Bonus)+(Maide
#  n_Bonus)+(Economy_Bonus)+(DotBalls×1)
# # Step 5: Player-Wise Average Fantasy Point
# avg_fp = df.groupby('Player_Name')['FPbowling'].mean().reset_index(name='avgfpbowling')
# df = df.merge(avg_fp, on='Player_Name', how='left')
# # Step 6: Merge Venue-Level Stats
# df = df.merge(
# venue_stats.rename(columns={
# 'wickets_taken': 'total_wickets_at_venue',
# 'balls_bowled': 'total_balls_bowled_at_venue',
# 'runs_conceded': 'total_runs_conceded_at_venue'
# })[
# ['Player_Name', 'venue', 'total_wickets_at_venue', 'total_balls_bowled_at_venue',
# 'total_runs_conceded_at_venue', 'Avg_Wickets_at_Venue', 'VPS_Bowling',
# 'Expected_Wickets_per_Match', 'Expected_Balls_Bowled_per_Match',
# 'Expected_Runs_Conceded_per_Match', 'Expected_Economy_Rate_per_Match',
# 'Expected_Bowling_Strike_Rate_per_Match']
# ],
# on=['Player_Name', 'venue'],
# how='left'
# )
# # Step 7: Export
# df.to_excel("bowling_final_stats.xlsx", index=False)
# print("
#  ✔
#  Bowling fantasy stats generated successfully!")
#Venue Index calculations
# ## base_features = [
#     "total_runs_conceded",     # Total runs given
#     "total_balls_y",           # Total balls bowled
#     "batting_strike_rate",     # Team's batting strike rate
#     "bowling_economy",         # Economy rate of bowling
#     "bowling_strike_rate",     # Strike rate of bowling
#     "total_wickets",           # Total wickets taken
#     "Venue",                   # Stadium name (for grouping)
#     "Matches Played"           # Match count per venue
# ]

# ## derived_features = [
#     "avg_runs_per_over",                 # Derived intermediate stat
#     "Calculated_Batting_Index",          # Batting index from formula
#     "Calculated_Bowling_Index",          # Bowling index from formula
#     "Matchwise_Batting_Index_Change",    # Match-to-match batting index change
#     "Matchwise_Bowling_Index_Change"     # Match-to-match bowling index change
# ]

# ## Code
# import pandas as pd

# # Load dataset
# df = pd.read_csv("stadium_data_final.csv")

# # Ensure relevant columns are numeric
# df["batting_strike_rate"] = pd.to_numeric(df["batting_strike_rate"], errors="coerce")
# df["total_runs_conceded"] = pd.to_numeric(df["total_runs_conceded"], errors="coerce")
# df["total_balls_y"] = pd.to_numeric(df["total_balls_y"], errors="coerce")
# df["total_wickets"] = pd.to_numeric(df["total_wickets"], errors="coerce")
# df["bowling_economy"] = pd.to_numeric(df["bowling_economy"], errors="coerce")
# df["bowling_strike_rate"] = pd.to_numeric(df["bowling_strike_rate"], errors="coerce")

# # ➤ Derived Feature 1: Average runs per over
# df["avg_runs_per_over"] = df["total_runs_conceded"] / (df["total_balls_y"] / 6)

# # ➤ Derived Feature 2: Batting Index Calculation
# df["Calculated_Batting_Index"] = (
#     df["avg_runs_per_over"] * df["batting_strike_rate"] * df["total_runs_conceded"]
# ) / 100000

# # ➤ Derived Feature 3: Bowling Index Calculation
# df["Calculated_Bowling_Index"] = (
#     (df["total_wickets"] / df["bowling_economy"]) * 10
# ) / df["bowling_strike_rate"]

# # ➤ Sort for match-wise changes
# df.sort_values(by=["Venue", "Matches Played"], inplace=True)

# # ➤ Derived Feature 4 & 5: Match-wise changes in indices
# df["Matchwise_Batting_Index_Change"] = df.groupby("Venue")["Calculated_Batting_Index"].diff().fillna(0)
# df["Matchwise_Bowling_Index_Change"] = df.groupby("Venue")["Calculated_Bowling_Index"].diff().fillna(0)

# # Save if needed
# df.to_csv("stadium_data_with_indices.csv", index=False)

# Normalisation and standardisation
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# # Load batting and bowling data
# batting_df = pd.read_excel("batting_final_stats.xlsx")   # Replace with your actual filename
# bowling_df = pd.read_excel("bowling_final_stats.xlsx")   # Replace with your actual filename
# # Columns to standardize (batting)
# batting_numeric_cols = [
# 'total_runs', 'balls_faced', 'sixes', 'fours', 'strike_rate',
# 'Boundary_Percentage', 'PPS_Batting', 'Matches_at_Venue', 'Avg_Runs_at_Venue',
# 'VPS_Batting', 'total_runs_at_venue', 'total_balls_faced_at_venue',
# 'total_sixes_at_venue', 'total_fours_at_venue',
# 'Expected_Runs_per_Match', 'Expected_Balls_Faced_per_Match',
# 'Expected_Sixes_per_Match', 'Expected_Fours_per_Match',
# 'Expected_Strike_Rate_per_Match', 'Expected_Boundary_Percentage_per_Match',
# '25R_Bonus', '50R_Bonus', '75R_Bonus', '100R_Bonus', 'Duck_Penalty',
# 'SR', 'SR_Bonus', 'Avg_Fantasy_Points_Phase_Batting'  # if exists
# ]
# # Columns to standardize (bowling)
# bowling_numeric_cols = [
# 'wickets_taken', 'balls_bowled', 'runs_conceded', 'economy_rate',
# 'bowling_strike_rate', 'PPS_Bowling', 'Matches_at_Venue',
# 'Avg_Wickets_at_Venue', 'VPS_Bowling', 'total_wickets_at_venue',
# 'total_balls_bowled_at_venue', 'total_runs_conceded_at_venue',
# 'Expected_Wickets_per_Match', 'Expected_Balls_Bowled_per_Match',
# 'Expected_Runs_Conceded_per_Match', 'Expected_Economy_Rate_per_Match',
# 'Expected_Bowling_Strike_Rate_per_Match', '3W_Bonus', '4W_Bonus',
# '5W_Bonus', 'Maiden_Bonus', 'ER', 'Economy_Bonus',
# 'FPbowling', 'avgfpbowling'
# ]
# # Apply StandardScaler
# scaler = StandardScaler()
# # Make a copy to avoid touching original data
# batting_scaled_df = batting_df.copy()
# bowling_scaled_df = bowling_df.copy()
# # Fit & transform
# batting_scaled_df[batting_numeric_cols] =
# scaler.fit_transform(batting_scaled_df[batting_numeric_cols])
# bowling_scaled_df[bowling_numeric_cols] =
# scaler.fit_transform(bowling_scaled_df[bowling_numeric_cols])
# # Save standardized versions
# batting_scaled_df.to_excel("batting_standardized.xlsx", index=False)
# bowling_scaled_df.to_excel("bowling_standardized.xlsx", index=False)
# print(" Standardized both batting and bowling datasets!")
# Calculating average of fpbatting from batter dataset:
# import pandas as pd
# # Load your dataset
# df = pd.read_csv('/content/batter_final_standardised.csv') # Replace with your actual file name
# # Group by 'Player' and calculate the average of 'FPbatting'
# average_fpbatting = df.groupby('Player_Name')['FPbatting'].mean().reset_index()
# # Optional: round to 4 decimal places
# average_fpbatting['FPbatting'] = average_fpbatting['FPbatting'].round(4)
# # Sort by FPbatting descending (optional)
# average_fpbatting = average_fpbatting.sort_values(by='FPbatting', ascending=False)
# # Display or save the result
# print(average_fpbatting)
# # Save to a new CSV file (optional)
# average_fpbatting.to_csv('average_FPbatting_per_player.csv', index=False)
# Calculating average of fpbowling from bowler dataset:
# import pandas as pd
# # Load your dataset
# df = pd.read_csv('/content/bowler_final_standardised.csv') # Replace with your actual file name
# df.columns = df.columns.str.strip()
# # Group by 'Player_Name' and calculate the average of 'FPbowling'
# average_fpbowling = df.groupby('Player_Name')['FPbowling'].mean().reset_index()
# # Optional: round to 4 decimal places
# average_fpbowling['FPbowling'] = average_fpbowling['FPbowling'].round(4)
# # Sort by FPbowling descending (optional)
# average_fpbowling = average_fpbowling.sort_values(by='FPbowling', ascending=False)
# # Display or save the result
# print(average_fpbowling)
# # Save to a new CSV file (optional)
# average_fpbowling.to_csv('average_FPbowling_per_player.csv', index=False)
# Merging the fpbatting and fpbowling with cricket data file:
# import pandas as pd
# # Load the two datasets
# df_bowling = pd.read_csv('average_FPbowling_per_player.csv')
# df_batting = pd.read_csv('average_FPbatting_per_player.csv') # Assuming you have this file for
# FPbatting
# df_cricket = pd.read_csv('cricket_data_processed.csv')
# # Strip any leading/trailing spaces in column names for both DataFrames
# df_bowling.columns = df_bowling.columns.str.strip()
# df_batting.columns = df_batting.columns.str.strip()
# df_cricket.columns = df_cricket.columns.str.strip()
# # Merge cricket data with both batting and bowling datasets
# merged_df = pd.merge(df_cricket, df_batting, on='Player_Name', how='left')
# merged_df = pd.merge(merged_df, df_bowling, on='Player_Name', how='left')
# # If there are players in cricket data not in batting data, fill 'FPbatting' with 0
# merged_df['FPbatting'].fillna(0, inplace=True)
# # If there are players in cricket data not in bowling data, fill 'FPbowling' with 0
# merged_df['FPbowling'].fillna(0, inplace=True)
# # Display the merged DataFrame (optional)
# print(merged_df.head())
# # Save the merged dataset to a new CSV file
# merged_df.to_csv('merged_cricket_data_with_fp_batting_bowling.csv', index=False)
# Calculating finalfp in the merged dataset:
# import pandas as pd
# # Load the merged dataset
# df = pd.read_csv('merged_cricket_data_with_fp_batting_bowling.csv')
# # Strip any leading/trailing spaces in column names (if needed)
# df.columns = df.columns.str.strip()
# # Calculate finalfp using the provided formula
# df['finalfp'] = 0.5 * df['Total_FP'] + 0.3 * df['FPbatting'] + 0.2 * df['FPbowling']
# # Display the DataFrame with the new 'finalfp' column (optional)
# print(df.head())
# # Save the updated dataset with finalfp to a new CSV file
# df.to_csv('merged_cricket_data_with_finalfp.csv', index=False)

# ----------------------------------------------------------------
# MODEL TRAINING CODE
# ----------------------------------------------------------------
# Training Phase:
# import pandas as pd
# import numpy as np
# import re
# from sklearn.ensemble import VotingRegressor, RandomForestRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# def convert_google_sheet_url(url):
# pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/.*)?'
# replacement = r'https://docs.google.com/spreadsheets/d/\1/export?format=csv'
# return re.sub(pattern, replacement, url)
# # --- Load files ---
# input_excel = pd.ExcelFile('/content/SquadPlayerNames_IndianT20League.xlsx')
# sheet_names = input_excel.sheet_names
# playing11_df = input_excel.parse(sheet_names[-1]) # Load last sheet
# schedule_df = pd.read_csv('/content/IPL_2025_Schedule_Finalll.csv')
# player_stats = pd.read_csv('/content/Player_Stats_Final.csv')
# recent_form_url =
# convert_google_sheet_url("https://docs.google.com/spreadsheets/d/1YtbjJ9UyKu7jo-fHKYzhA
#  ZS0iilJaOjL0jQQx1Aprac/edit?usp=sharing")
# recent_form = pd.read_csv(recent_form_url)
# # Match number from sheet index
# match_number = sheet_names.index(sheet_names[-1]) + 1
# match_row = schedule_df[schedule_df['Match Number'] == match_number].iloc[0]
# city = match_row['City']
# stadium = match_row['Stadium']
# # Filter venue-specific stats from player_stats
# venue_stats = player_stats.copy()
# venue_stats.fillna(0, inplace=True)
# # --- Feature selection ---
# features = ['FP_Batting', 'FP_Bowling', 'FP_Fielding', 'FPbatting','FPbowling']
# # --- Train model ---
# X_train = venue_stats[features]
# y_train = venue_stats['Final FP']
# rf = RandomForestRegressor(n_estimators=300, max_depth=10, min_samples_split=5,
# min_samples_leaf=3, random_state=42)
# xgb = XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.1, random_state=42)
# model = VotingRegressor([('rf', rf), ('xgb', xgb)])
# model.fit(X_train, y_train)
# pred = model.predict(X_train)
# print("MAE:", mean_absolute_error(y_train, pred))
# print("RMSE:", np.sqrt(mean_squared_error(y_train, pred)))
# print("R2:", r2_score(y_train, pred))



#-------------------------------------
# ✅ Fantasy Team Prediction Script
#-------------------------------------

import pandas as pd
import numpy as np
import joblib
import requests
import re
from datetime import datetime
!pip install rapidfuzz
!pip install selenium
!pip install webdriver-manager
!pip install selenium webdriver-manager beautifulsoup4

import rapidfuzz
from rapidfuzz import process, fuzz

import tempfile
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from sklearn.preprocessing import StandardScaler
import os
from webdriver_manager.chrome import ChromeDriverManager
import shutil

!wget -q -O google-chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome.deb || apt-get -fy install

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    # Try to find chrome binary
    chrome_path = shutil.which("google-chrome") or shutil.which("chrome") or shutil.which("chromium") or "/usr/bin/google-chrome"

    chrome_options.binary_location = chrome_path

    user_data_dir = tempfile.mkdtemp()
    chrome_options.add_argument(f"--user-data-dir={user_data_dir}")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver


def scrape_and_process_recent_form_data():
    # Setup Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.binary_location = "/usr/bin/google-chrome"

    # Setup user data dir
    user_data_dir = tempfile.mkdtemp()
    chrome_options.add_argument(f"--user-data-dir={user_data_dir}")

    # Setup WebDriver
    service = Service("/usr/bin/chromedriver")
    #driver = webdriver.Chrome(service=service, options=chrome_options)
    driver = setup_driver()

    url = "https://www.howstat.com/Cricket/Statistics/IPL/SeriesAnalysis.asp?s=2025"
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    def extract_table(div_id):
        div = soup.find('div', {'id': div_id})
        table = div.find('table', {'class': 'TableLined'})
        headers = [th.get_text(strip=True) for th in table.find_all('tr')[0].find_all(['th', 'td'])]
        rows = []
        for tr in table.find_all('tr')[1:]:
            cols = tr.find_all('td')
            if len(cols) == len(headers):
                rows.append([td.get_text(strip=True) for td in cols])
        return pd.DataFrame(rows, columns=headers)

    batting_df = extract_table("bat")
    bowling_df = extract_table("bowl")

    def preprocess_batting_df(df):

      df = df.copy()
      df['team_runs_float'] = df['% Team Runs'].astype(str).str.extract(r'([\d.]+)').astype(float)
      if 'HS' in df.columns:
        df['HS'] = df['HS'].astype(str).str.replace('*', '', regex=False)
        df['HS'] = pd.to_numeric(df['HS'], errors='coerce')

      numeric_cols = ['Runs', 'Avg', 'S/R', '50s', '100s', '0s', 'HS', 'Ca', 'St', 'Inns', 'Mat', 'NO', 'team_runs_float']
      df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
      for col in numeric_cols:
        if pd.api.types.is_numeric_dtype(df[col]):
          df[col] = df[col].fillna(df[col].mean())

      #df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

      # Add recent_form_batting_score
      if set(['Runs','Avg','S/R','50s','100s','0s','HS','% Team Runs','Ca','St','Inns','Mat','NO']).issubset(df.columns):
        df['recent_form_batting_score'] = (
            0.30 * df['Runs'] +
            0.15 * df['Avg'] +
            0.10 * df['S/R'] +
            0.10 * (df['50s'] + df['100s']) -
            0.10 * df['0s'] +
            0.05 * df['HS'] +
            0.05 * df['team_runs_float'] +
            0.05 * (df['Ca'] + df['St']) +
            0.10 * (df['Inns'] + df['Mat'] + df['NO']) / 3
          )

      numeric_cols = df.select_dtypes(include='number').columns
      scaler = StandardScaler()
      df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
      return df

    def preprocess_bowling_df(df):
      df = df.copy()
      exclude = ['Player', 'Team', 'Best']
      for col in df.columns:
          if col not in exclude:
              df[col] = pd.to_numeric(df[col], errors='coerce')
      if 'Best' in df.columns:
          df['Best'] = df['Best'].astype(str).str.extract(r'(\d+)', expand=False)
          df['Best'] = pd.to_numeric(df['Best'], errors='coerce')
      for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
          df[col] = df[col].fillna(df[col].mean())
      #df = df.fillna(df.mean())

      # Add bowling score before normalization
      if set(['W','Avg','E/R','S/R','4w','Best','M','Inns','Mat','O']).issubset(df.columns):
        df['recent_form_bowling_score'] = (
            0.30 * df['W'] +
            0.15 * (1 - df['Avg']) +
            0.10 * (1 - df['E/R']) +
            0.10 * (1 - df['S/R']) +
            0.10 * df['4w'] +
            0.05 * df['Best'] +
            0.05 * df['M'] +
            0.15 * (df['Inns'] + df['Mat'] + df['O']) / 3
        )

      from sklearn.preprocessing import StandardScaler

      numeric_cols = df.select_dtypes(include='number').columns
      scaler = StandardScaler()
      df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

      return df

    recent_form_bat = preprocess_batting_df(batting_df)
    recent_form_bowl = preprocess_bowling_df(bowling_df)

    return recent_form_bat, recent_form_bowl


# -----------------------------------------
# MANUAL MAPPINGS
# -----------------------------------------

manual_recent_form_mapping = {
    'A Badoni': 'Ayush Badoni',
    'A D Russell': 'Andre Russell',
    'A K Markram': 'Aiden Markram',
    'A M Rahane': 'Ajinkya Rahane',
    'A Manohar': 'Abhinav Manohar',
    'A Porel': 'Abishek Porel',
    'A R Patel': 'Axar Patel',
    'A Zampa': 'Adam Zampa',
    'Abdul Samad': 'Abdul Samad',
    'Abhishek Sharma': 'Abhishek Sharma',
    'Akash Deep': 'Akash Deep',
    'Angkrish Raghuvanshi': 'Angkrish Raghuvanshi',
    'Anshul Kamboj': 'Anshul Kamboj',
    'Arshad Khan': 'Arshad Khan',
    'Arshdeep Singh': 'Arshdeep Singh',
    'Ashutosh Sharma': 'Ashutosh Sharma',
    'Avesh Khan': 'Avesh Khan',
    'Azmatullah Omarzai': 'Azmatullah Omarzai',
    'A Kumar': 'Ashwani Kumar',
    'A M Hardie' : 'Aaron Hardie',
    'Abhinandan Singh' : 'Abhinandan Singh',
    'A Madhwal' : 'Akash Madhwal',
    'Akash Singh' : 'Akash Singh',
    'A Siddharth' : 'Andre Siddharth',
    'A Nortje' : 'Anrich Nortje',
    'Anuj Rawat' : 'Anuj Rawat',
    'AS Roy' : 'Anukul Sudhakar Roy',
    'A Mandal' : 'Ajay Mandal',
    'A S Tendulkar' : 'Arjun Tendulkar',
    'Arshin Kulkarni' : 'Arshin Kulkarni',
    'Ashok Sharma' : 'Ashok Sharma',
    'Ayush Mhatre' : 'Ayush Mhatre',
    'A Taide' : 'Atharva Taide',
    'B Kumar': 'Bhuvneshwar Kumar',
    'B Jacobs' : 'Bevon Jacobs',
    'C V Varun': 'Varun Chakravarthy',
    'C Sakariya' : 'Chetan Sakariya',
    'C Bosch' : 'Corbin Bosch',
    'D A Miller': 'David Miller',
    'D C Jurel': 'Dhruv Jurel',
    'D J Hooda': 'Deepak Hooda',
    'D L Chahar': 'Deepak Chahar',
    'D P Conway': 'Devon Conway',
    'D Padikkal': 'Devdutt Padikkal',
    'Digvesh Rathi': 'Digvesh Singh',
    'D G Nalkande' : 'Darshan Nalkande',
    'D Ferreira' : 'Donovan Ferreira',
    'D Brevis' : 'Dewald Brevis',
    'D Shanaka' : 'Dasun Shanaka',
    'Eshan Malinga': 'Eshan Malinga',
    'F du Plessis': 'Faf du Plessis',
    'Fazalhaq Farooqi': 'FazalHaq Farooqi',
    'G J Maxwell': 'Glenn Maxwell',
    'G Coetzee' : 'Gerald Coetzee',
    'Gurnoor Brar' :  'Gurnoor Brar Singh',
    'Gurjapneet Singh' : 'Gurjapneet Singh',
    'H H Pandya': 'Hardik Pandya',
    'H Klaasen': 'Heinrich Klaasen',
    'H P Rana': 'Harshit Rana',
    'H V Patel': 'Harshal Patel',
    'Himmat Singh': 'Himmat Singh',
    'Harnoor Pannu' : 'Harnoor Singh Pannu',
    'Harpreet Brar' : 'Harpreet Brar',
    'I Sharma': 'Ishant Sharma',
    'Ishan Kishan': 'Ishan Kishan',
    'I D N Thushara': 'Nuwan Thushara',
    'J C Archer': 'Jofra Archer',
    'J C Buttler': 'Jos Buttler',
    'J D Unadkat': 'Jaydev Unadkat',
    'J Fraser-McGurk': 'Jake Fraser-McGurk',
    'J J Bumrah': 'Jasprit Bumrah',
    'J M Sharma': 'Jitesh Sharma',
    'J Overton': 'Jamie Overton',
    'J R Hazlewood': 'Josh Hazlewood',
    'J P Inglis': 'Josh Inglis',
    'J G Bethell' : 'Jacob Bethell',
    'J Yadav': 'Jayant Yadav',
    'J G Bethell' : 'Jacob Bethell',
    'J Yadav' :'Jayant Yadav',
    'K H Pandya': 'Krunal Pandya',
    'K K Ahmed': 'Khaleel Ahmed',
    'K Khejroliya': 'Kulwant Khejroliya',
    'K L Rahul': 'Lokesh Rahul',
    'K Rabada': 'Kagiso Rabada',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'Kumar Kartikeya': 'Kumar Kartikeya',
    'K K Nair': 'Karun Nair',
    'K V Sharma': 'Karn Sharma',
    'K L Nagarkoti' : 'Kamlesh Nagarkoti',
    'Karim Janat': 'Karim Janat',
    'K Srikkanth': 'Krishnamachari Srikkanth',
    'K R Sen' : 'Kuldeep Sen',
    'Kumar Kushagra': 'Kumar Kushagra',
    'K S Rathore': 'Kunal Singh Rathore',
    'K T Maphaka': 'Kwena Maphaka',
    'L H Ferguson': 'Lockie Ferguson',
    'L S Livingstone': 'Liam Livingstone',
    'L Ngidi': 'Lungi Ngidi',
    'Luvnith Sisodia': 'Luvnith Sisodia',
    'M A Starc': 'Mitchell Starc',
    'M J Santner': 'Mitchell Santner',
    'M Jansen': 'Marco Jansen',
    'M K Pandey': 'Manish Pandey',
    'M M Ali': 'Moeen Ali',
    'M M Sharma': 'Mohit Sharma',
    'M M Theekshana': 'Maheesh Theekshana',
    'M P Stoinis': 'Marcus Stoinis',
    'M Pathirana': 'Matheesha Pathirana',
    'M Prasidh Krishna': 'Prasidh Krishna',
    'M R Marsh': 'Mitchell Marsh',
    'M S Dhoni': 'MS Dhoni',
    'M Siddharth': 'Manimaran Siddharth',
    'Mohammed Shami': 'Mohammed Shami',
    'Mohammed Siraj': 'Mohammed Siraj',
    'Mujeeb Ur Rahman': 'Mujeeb-ur-Rahman',
    'Mukesh Choudhary': 'Mukesh Choudhary',
    'M Kumar': 'Mukesh Kumar',
    'M K Lomror': 'Mahipal Lomror',
    'Manav Suthar': 'Manav Suthar',
    'Manoj Bhandage': 'Manoj Bhandage',
    'Madhav Tiwari': 'Madhav Tiwari',
    'Manvanth Kumar L': 'Manvanth Kumar L',
    'M P Breetzkee': 'Matthew Breetzkee',
    'M Markande': 'Mayank Markande',
    'M P Yadav': 'Mayank Yadav',
    'M Rathee': 'Mohit Rathee',
    'Musheer Khan': 'Musheer Khan',
    'N Ellis': 'Nathan Ellis',
    'N K Reddy': 'K Nitish Reddy',
    'N Pooran': 'Nicholas Pooran',
    'N Rana': 'Nitish Rana',
    'N Wadhera': 'Nehal Wadhera',
    'Naman Dhir': 'Naman Dhir',
    'Noor Ahmad': 'Noor Ahmad',
    'Nishant Sindhu': 'Nishant Sidhu',
    'P Arya': 'Priyansh Arya',
    'P D Salt': 'Philip Salt',
    'P H K D Mendis': 'Kamindu Mendis',
    'P J Cummins': 'Pat Cummins',
    'P V S Raju': 'Satyanarayana Raju',
    'P W A Mulder': 'Wiaan Mulder',
    'P W H de Silva': 'Wanindu Hasaranga',
    'P Yadav': 'Prince Yadav',
    'Prabhsimran Singh': 'Prabhsimran Singh',
    'P Dubey': 'Pravin Dubey',
    'Pyla Avinash': 'Pyla Avinash',
    'P V D Chameera' : 'Dushmantha Chameera',
    'Q de Kock': 'Quinton de Kock',
    'R A Bawa': 'Raj Angad Bawa',
    'R A Jadeja': 'Ravindra Jadeja',
    'R A Tripathi': 'Rahul Tripathi',
    'R Ashwin': 'Ravichandran Ashwin',
    'R D Gaikwad': 'Ruturaj Gaikwad',
    'R D Rickelton': 'Ryan Rickelton',
    'R G Sharma': 'Rohit Sharma',
    'R M Patidar': 'Rajat Patidar',
    'R Minz': 'Robin Minz',
    'R Parag': 'Riyan Parag',
    'R R Pant': 'Rishabh Pant',
    'R Ravindra': 'Rachin Ravindra',
    'R S Kishore': 'Ravisrinivasan Sai Kishore',
    'R Tewatia': 'Rahul Tewatia',
    'Ramandeep Singh': 'Ramandeep Singh',
    'Rashid Khan': 'Rashid-Khan',
    'Rasikh Salam': 'Rasikh Salam',
    'Ravi Bishnoi': 'Ravi Bishnoi',
    'Rinku Singh': 'Rinku Singh',
    'R D Chahar' : 'Rahul Chahar',
    'Rahmanullah Gurbaz' : 'Rahmanullah Gurbaz',
    'R Hangargekar' : 'Rajvardhan Hangargekar',
    'R J W Topley' : 'Reece Topley',
    'R Shepherd' : 'Romario Shepherd',
    'R Powell' : 'Rovman Powell',
    'R Dhawan' : 'Rishi Dhawan',
    'S A Yadav': 'Suryakumar Yadav',
    'S Dube': 'Shivam Dube',
    'S E Rutherford': 'Sherfane Rutherford',
    'S H Johnson': 'Spencer Johnson',
    'S M Curran': 'Sam Curran',
    'S N Thakur': 'Shardul Thakur',
    'S O Hetmeyer': 'Shimron Hetmyer',
    'S P Narine': 'Sunil Narine',
    'S Rizvi': 'Sameer Rizvi',
    'S S Iyer': 'Shreyas Iyer',
    'S Sharma': 'Suyash Sharma',
    'S Shedge': 'Suryansh Shedge',
    'S V Samson': 'Sanju Samson',
    'Sai Sudharsan': 'Sai Sudharsan',
    'Sandeep Sharma': 'Sandeep Sharma',
    'Shahbaz Ahmed': 'Shahbaz Ahmed',
    'Shahrukh Khan': 'Shahrukh Khan',
    'Shubham Dubey': 'Shubham Dubey',
    'Shubman Gill': 'Shubman Gill',
    'Shaik Rasheed': 'Shaik Rasheed',
    'S Singh': 'Simarjeet- Singh',
    'S Singh': 'Shashank Singh',
    'Sachin Baby' : 'Sachin Baby',
    'S Gopal' : 'Shreyas Gopal',
    'Swapnil Singh' : 'Swapnil Singh',
    'S Ravichandran' : 'Smaran Ravichandran',
    'Shrijith Krishnan' : 'KL Shrijith',
    'Shariz Ahmad' : 'Shariz Ahmad',
    'Swastik Chikara' : 'Swastik Chikara',
    'T A Boult': 'Trent Boult',
    'T H David': 'Tim David',
    'T M Head': 'Travis Head',
    'T Stubbs': 'Tristan Stubbs',
    'T U Deshpande': 'Tushar Deshpande',
    'Tilak Varma': 'Tilak Varma',
    'T Natarajan' : 'T Natarajan',
    'Tripurana Vijay' : 'Tripurana Vijay',
    'V G Arora': 'Vaibhav Arora',
    'V Kohli': 'Virat Kohli',
    'V Nigam': 'Vipraj Nigam',
    'V Puthur': 'Vignesh Puthur',
    'V R Iyer': 'Venkatesh Iyer',
    'V Shankar': 'Vijay Shankar',
    'Vijaykumar Vyshak': 'Vyshak Vijaykumar',
    'V Suryavanshi' : 'Vaibhav Suryavanshi',
    'Vishnu Vinod' : 'Vishnu Vinod',
    'Vansh Bedi' : 'Vansh Bedi',
    'W G Jacks': 'Will Jacks',
    'Washington Sundar': 'Washington Sundar',
    'X C Bartlett': 'Xavier Bartlett',
    'Y B K Jaiswal': 'Yashasvi Jaiswal',
    'Y Dayal': 'Yash Dayal',
    'Y S Chahal': 'Yuzvendra Chahal',
    'Y Thakur': 'Yash Thakur',
    'Yudhvir Singh': 'Yudhvir Singh Charak',
    'Yuvraj Chaudhary' : 'Yuvraj Chaudhary',
    'Z Ansari': 'Zeeshan Ansari',
    'A Verma': 'Aniket Verma',
    'R Ghosh': 'Ramakrishna Ghosh',
    'PVD Chameera': 'Dushmantha Chameera',
    'GD Phillips': 'Glenn Phillips',
    'S Joseph': 'Shamar Joseph',
    'A Juyal': 'Aryan Juyal',
    'Mohsin Khan': 'Mohsin Khan',
  }

# -----------------------------------------
# UTILITY FUNCTIONS
# -----------------------------------------


def convert_google_sheet_url(url):
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/.*)?'
    replacement = r'https://docs.google.com/spreadsheets/d/\1/export?format=csv'
    return re.sub(pattern, replacement, url)

def fetch_weather(city):
    # Static city to coordinates mapping (Add more as needed)
    city_coordinates = {
        'Delhi': {'latitude': 28.61, 'longitude': 77.23},
        'Mumbai': {'latitude': 19.07, 'longitude': 72.87},
        'Chennai': {'latitude': 13.08, 'longitude': 80.27},
        'Kolkata': {'latitude': 22.57, 'longitude': 88.36},
        'Bengaluru': {'latitude': 12.97, 'longitude': 77.59},
        'Hyderabad': {'latitude': 17.38, 'longitude': 78.48},
        'Ahmedabad': {'latitude': 23.03, 'longitude': 72.58},
        'Lucknow': {'latitude': 26.85, 'longitude': 80.95},
        'Jaipur': {'latitude': 26.91, 'longitude': 75.79},
        'Dharamsala': {'latitude': 32.22, 'longitude': 76.32},
        'Visakhapatnam': {'latitude': 17.69, 'longitude': 83.22},
        'Chandigarh': {'latitude': 30.74, 'longitude': 76.78},
        'Guwahati': {'latitude': 26.12, 'longitude': 91.71}
    }

    try:
        # Default to Delhi if city not found
        coords = city_coordinates.get(city, city_coordinates['Delhi'])
        params = {
            'latitude': coords['latitude'],
            'longitude': coords['longitude'],
            'hourly': 'temperature_2m,precipitation,relative_humidity_2m,cloud_cover,dew_point_2m,wind_speed_10m',
            'timezone': 'auto'
        }
        res = requests.get("https://api.open-meteo.com/v1/forecast", params=params)
        data = res.json()
        return {
            'precipitation': np.mean(data['hourly']['precipitation']),
            'humidity': np.mean(data['hourly']['relative_humidity_2m']),
            'wind_speed': np.mean(data['hourly']['wind_speed_10m']),
            'cloud_cover': np.mean(data['hourly']['cloud_cover']),
            'dew_point': np.mean(data['hourly']['dew_point_2m'])
        }
    except:
        return {'precipitation': 0.2, 'humidity': 60, 'wind_speed': 4, 'cloud_cover': 40, 'dew_point': 16}

def calculate_weather_impact(w):
    return (
        0.35 * w['precipitation'] +
        0.25 * w['humidity'] +
        0.15 * w['wind_speed'] +
        0.15 * w['cloud_cover'] +
        0.10 * w['dew_point']
    )


def get_match_details_from_last_sheet(excel_file_path: str, schedule_csv_path: str):

    try:
        # Load and clean the schedule CSV
        schedule_df = pd.read_csv(schedule_csv_path)
        schedule_df.columns = schedule_df.columns.str.strip()

        # Get last sheet name from the Excel file
        xls = pd.ExcelFile(excel_file_path)
        last_sheet_name = xls.sheet_names[-1]

        # Extract match number from sheet name like 'Match_23'
        match_number = int(re.search(r'Match_(\d+)', last_sheet_name).group(1))

        # Locate the row for that match
        row = schedule_df[schedule_df['Match Number'] == match_number]

        if not row.empty:
            row = row.iloc[0]
            result = {
                'match_number': match_number,
                'venue': row['Stadium'],
                'city': row['City'],
                'batting_friendly_index': row['Batting-Friendly Index(matchwise)'],
                'bowling_friendly_index': row['Bowling-Friendly Index(matchwise)'],
                'rpo_zscore': row['RPO_zscore'],
                'rpw_zscore': row['RPW_zscore']
            }
            return result
        else:
            return None

    except Exception:
        return None


def get_ordinal(n):
    # Return the ordinal representation (1st, 2nd, 3rd, 4th, etc.)
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"


def fetch_toss_info(match_number, apikey):
    try:
        # Step 1: Fetch all matches
        url = f"https://api.cricapi.com/v1/currentMatches?apikey={apikey}"
        response = requests.get(url)
        response.raise_for_status()
        matches_data = response.json().get("data", [])

        match_id = None
        #match_keyword = f"{match_number}th Match"
        match_keyword = f"{get_ordinal(match_number)} Match"  # e.g., 31st Match

        for match in matches_data:
            if (match.get("matchType") == "t20" and
                match_keyword in match.get("name", "")):
                match_id = match.get("id")
                break

        if not match_id:
            return None, None, None

        # Step 2: Fetch match info using match_id
        info_url = f"https://api.cricapi.com/v1/match_info?apikey={apikey}&id={match_id}"
        match_info = requests.get(info_url).json()

        toss_winner = match_info['data'].get('tossWinner')
        toss_decision = match_info['data'].get('tossChoice')

        if not toss_winner or not toss_decision:
            return match_id, None, None

        return match_id, toss_winner, toss_decision

    except Exception:
        return None, None, None


def compute_interaction_score(row, all_players_df, pvp_df):
  player = row['Player Name']
  team = row['Team']
  player_type = row['Player Type']

  # Identify opposing players (from other team only)
  opponents = all_players_df[all_players_df['Team'] != team]['Player Name'].tolist()

  scores = []
  for opponent in opponents:
      if player_type == 'BAT':
        # Check batter-bowler pair
        cond = (pvp_df['mapped bat players'] == player) & (pvp_df['mapped bowl players'] == opponent)
      else:
        # Check bowler-batter pair
        cond = (pvp_df['mapped bowl players'] == player) & (pvp_df['mapped bat players'] == opponent)

      subset = pvp_df[cond]
      if not subset.empty:
        scores.append(subset['interaction_score'].values[0])  # use first match value

  if scores:
    return np.mean(scores)
  else:
    return 0.2  # Default neutral score if no history

def compute_captain_score(row):
  """
    Computes a captain score to help rank players for Captain/Vice-Captain selection.
    Considers prediction score, recent form, role-based bias, and batting/bowling order.

    Parameters:
    - row (pd.Series): Row of player data.

    Returns:
    - float: Captain score for the player.
    """
  try:
    # Use .fillna(0) approach for the row to safely access numeric values
    row = row.fillna(0)

    score = 0.0

    # Base prediction score
    score += 0.5 * row['prediction_score']

    # Recent form scores
    score += 0.2 * row['recent_form_batting_score']
    score += 0.2 * row['recent_form_bowling_score']

    # Role-based bias
    if row['Player Type'] == 'BAT':
      score += 1
    elif row['Player Type'] == 'ALL':
      score += 1

    # Penalize lower-order batters or bowlers who might not get full quota
    if 'lineupOrder' in row and not pd.isna(row['lineupOrder']):
      lineup_order = row['lineupOrder']
      if row['Player Type'] in ['BAT', 'WK'] and lineup_order > 5:
        score -= 1

    return score

  except Exception:
    return 0  # Fail-safe fallback

# -----------------------------------------
# LOAD TRAINED MODEL AND DATASETS
# -----------------------------------------
model = joblib.load("fantasy_prediction_model.pkl")
player_stats = pd.read_csv("Player_Stats.csv")
venue_df = pd.read_csv("IPL_2025_Schedule_Final.csv")
player_vs_player_df=pd.read_csv("player_vs_player_data.csv")


# Get recent form data dynamically via web scraping
recent_form_bat, recent_form_bowl = scrape_and_process_recent_form_data()

# Standardize column names
def clean_name(name):
  """
    Standardizes player or column names by:
    - Converting to uppercase
    - Stripping extra whitespace

    Parameters:
    - name (str): Input name

    Returns:
    - str: Cleaned name
    """
  return str(name).upper().strip()

# -----------------------------------------
# MAIN PREDICTION FUNCTION
# -----------------------------------------
def predict_fantasy_team(input_file, venue_name, toss_decision, toss_winner, city):
  # Standardize player names across datasets
  player_stats['Player_Name'] = player_stats['Player_Name'].apply(clean_name)
  recent_form_bat['Player'] = recent_form_bat['Player'].apply(clean_name)
  recent_form_bowl['Player'] = recent_form_bowl['Player'].apply(clean_name)

  input_file_df = pd.read_excel(input_file)
  # Load all sheet names
  sheet_names = pd.ExcelFile(input_file).sheet_names
  # Read the last sheet
  input_df = pd.read_excel(input_file, sheet_name=sheet_names[-1])

  # Filter playing players
  input_df = input_df[input_df['IsPlaying'].str.upper() == 'PLAYING'].drop_duplicates(subset='Player Name')
  input_df['Player Name'] = input_df['Player Name'].apply(clean_name)

  # Match with historical stats (exact match only)
  # === Step 1: Merge exact matches ===
  merged = pd.merge(input_df, player_stats, left_on='Player Name', right_on='Player_Name', how='left')

  #marking new players
  merged['is_new_player'] = merged['Final FP'].isna().astype(int)

  # === Optional Preview (ffor player_stats) ===
  missing_player_stats = merged[
      merged['Final FP'].isna()
  ]

  merged['Final FP'] = merged['Final FP'].fillna(0)

  #-----------Manual Mapping Dictionary---------------
  manual_recent_form_mapping = {
    'A Badoni': 'Ayush Badoni',
    'A D Russell': 'Andre Russell',
    'A K Markram': 'Aiden Markram',
    'A M Rahane': 'Ajinkya Rahane',
    'A Manohar': 'Abhinav Manohar',
    'A Porel': 'Abishek Porel',
    'A R Patel': 'Axar Patel',
    'A Zampa': 'Adam Zampa',
    'Abdul Samad': 'Abdul Samad',
    'Abhishek Sharma': 'Abhishek Sharma',
    'Akash Deep': 'Akash Deep',
    'Angkrish Raghuvanshi': 'Angkrish Raghuvanshi',
    'Anshul Kamboj': 'Anshul Kamboj',
    'Arshad Khan': 'Arshad Khan',
    'Arshdeep Singh': 'Arshdeep Singh',
    'Ashutosh Sharma': 'Ashutosh Sharma',
    'Avesh Khan': 'Avesh Khan',
    'Azmatullah Omarzai': 'Azmatullah Omarzai',
    'A Kumar': 'Ashwani Kumar',
    'A M Hardie' : 'Aaron Hardie',
    'Abhinandan Singh' : 'Abhinandan Singh',
    'A Madhwal' : 'Akash Madhwal',
    'Akash Singh' : 'Akash Singh',
    'A Siddharth' : 'Andre Siddharth',
    'A Nortje' : 'Anrich Nortje',
    'Anuj Rawat' : 'Anuj Rawat',
    'AS Roy' : 'Anukul Sudhakar Roy',
    'A Mandal' : 'Ajay Mandal',
    'A S Tendulkar' : 'Arjun Tendulkar',
    'Arshin Kulkarni' : 'Arshin Kulkarni',
    'Ashok Sharma' : 'Ashok Sharma',
    'A Taide' : 'Atharva Taide',
    'Ayush Mhatre' : 'Ayush Mhatre',
    'B Kumar': 'Bhuvneshwar Kumar',
    'B Jacobs' : 'Bevon Jacobs',
    'C V Varun': 'Varun Chakravarthy',
    'C Sakariya' : 'Chetan Sakariya',
    'C Bosch' : 'Corbin Bosch',
    'D A Miller': 'David Miller',
    'D C Jurel': 'Dhruv Jurel',
    'D J Hooda': 'Deepak Hooda',
    'D L Chahar': 'Deepak Chahar',
    'D P Conway': 'Devon Conway',
    'D Padikkal': 'Devdutt Padikkal',
    'Digvesh Rathi': 'Digvesh Singh',
    'D G Nalkande' : 'Darshan Nalkande',
    'D Ferreira' : 'Donovan Ferreira',
    'D Brevis' : 'Dewald Brevis',
    'D Shanaka' : 'Dasun Shanaka',
    'Eshan Malinga': 'Eshan Malinga',
    'F du Plessis': 'Faf du Plessis',
    'Fazalhaq Farooqi': 'FazalHaq Farooqi',
    'G J Maxwell': 'Glenn Maxwell',
    'G Coetzee' : 'Gerald Coetzee',
    'Gurnoor Brar' :  'Gurnoor Brar Singh',
    'Gurjapneet Singh' : 'Gurjapneet Singh',
    'H H Pandya': 'Hardik Pandya',
    'H Klaasen': 'Heinrich Klaasen',
    'H P Rana': 'Harshit Rana',
    'H V Patel': 'Harshal Patel',
    'Himmat Singh': 'Himmat Singh',
    'Harnoor Pannu' : 'Harnoor Singh Pannu',
    'Harpreet Brar' : 'Harpreet Brar',
    'I Sharma': 'Ishant Sharma',
    'Ishan Kishan': 'Ishan Kishan',
    'I D N Thushara': 'Nuwan Thushara',
    'J C Archer': 'Jofra Archer',
    'J C Buttler': 'Jos Buttler',
    'J D Unadkat': 'Jaydev Unadkat',
    'J Fraser-McGurk': 'Jake Fraser-McGurk',
    'J J Bumrah': 'Jasprit Bumrah',
    'J M Sharma': 'Jitesh Sharma',
    'J Overton': 'Jamie Overton',
    'J R Hazlewood': 'Josh Hazlewood',
    'J P Inglis': 'Josh Inglis',
    'J G Bethell' : 'Jacob Bethell',
    'J Yadav': 'Jayant Yadav',
    'J G Bethell' : 'Jacob Bethell',
    'J Yadav' :'Jayant Yadav',
    'K H Pandya': 'Krunal Pandya',
    'K K Ahmed': 'Khaleel Ahmed',
    'K Khejroliya': 'Kulwant Khejroliya',
    'K L Rahul': 'Lokesh Rahul',
    'K Rabada': 'Kagiso Rabada',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'Kumar Kartikeya': 'Kumar Kartikeya',
    'K K Nair': 'Karun Nair',
    'K V Sharma': 'Karn Sharma',
    'K L Nagarkoti' : 'Kamlesh Nagarkoti',
    'Karim Janat': 'Karim Janat',
    'K Srikkanth': 'Krishnamachari Srikkanth',
    'K R Sen' : 'Kuldeep Sen',
    'Kumar Kushagra': 'Kumar Kushagra',
    'K S Rathore': 'Kunal Singh Rathore',
    'K T Maphaka': 'Kwena Maphaka',
    'L H Ferguson': 'Lockie Ferguson',
    'L S Livingstone': 'Liam Livingstone',
    'L Ngidi': 'Lungi Ngidi',
    'Luvnith Sisodia': 'Luvnith Sisodia',
    'M A Starc': 'Mitchell Starc',
    'M J Santner': 'Mitchell Santner',
    'M Jansen': 'Marco Jansen',
    'M K Pandey': 'Manish Pandey',
    'M M Ali': 'Moeen Ali',
    'M M Sharma': 'Mohit Sharma',
    'M M Theekshana': 'Maheesh Theekshana',
    'M P Stoinis': 'Marcus Stoinis',
    'M Pathirana': 'Matheesha Pathirana',
    'M Prasidh Krishna': 'Prasidh Krishna',
    'M R Marsh': 'Mitchell Marsh',
    'M S Dhoni': 'MS Dhoni',
    'M Siddharth': 'Manimaran Siddharth',
    'Mohammed Shami': 'Mohammed Shami',
    'Mohammed Siraj': 'Mohammed Siraj',
    'Mujeeb Ur Rahman': 'Mujeeb-ur-Rahman',
    'Mukesh Choudhary': 'Mukesh Choudhary',
    'M Kumar': 'Mukesh Kumar',
    'M K Lomror': 'Mahipal Lomror',
    'Manav Suthar': 'Manav Suthar',
    'Manoj Bhandage': 'Manoj Bhandage',
    'Madhav Tiwari': 'Madhav Tiwari',
    'Manvanth Kumar L': 'Manvanth Kumar L',
    'M P Breetzkee': 'Matthew Breetzkee',
    'M Markande': 'Mayank Markande',
    'M P Yadav': 'Mayank Yadav',
    'M Rathee': 'Mohit Rathee',
    'Musheer Khan': 'Musheer Khan',
    'N Ellis': 'Nathan Ellis',
    'N K Reddy': 'K Nitish Reddy',
    'N Pooran': 'Nicholas Pooran',
    'N Rana': 'Nitish Rana',
    'N Wadhera': 'Nehal Wadhera',
    'Naman Dhir': 'Naman Dhir',
    'Noor Ahmad': 'Noor Ahmad',
    'Nishant Sindhu': 'Nishant Sidhu',
    'P Arya': 'Priyansh Arya',
    'P D Salt': 'Philip Salt',
    'P H K D Mendis': 'Kamindu Mendis',
    'P J Cummins': 'Pat Cummins',
    'P V S Raju': 'Satyanarayana Raju',
    'P W A Mulder': 'Wiaan Mulder',
    'P W H de Silva': 'Wanindu Hasaranga',
    'P Yadav': 'Prince Yadav',
    'Prabhsimran Singh': 'Prabhsimran Singh',
    'P Dubey': 'Pravin Dubey',
    'Pyla Avinash': 'Pyla Avinash',
    'P V D Chameera' : 'Dushmantha Chameera',
    'Q de Kock': 'Quinton de Kock',
    'R A Bawa': 'Raj Angad Bawa',
    'R A Jadeja': 'Ravindra Jadeja',
    'R A Tripathi': 'Rahul Tripathi',
    'R Ashwin': 'Ravichandran Ashwin',
    'R D Gaikwad': 'Ruturaj Gaikwad',
    'R D Rickelton': 'Ryan Rickelton',
    'R G Sharma': 'Rohit Sharma',
    'R M Patidar': 'Rajat Patidar',
    'R Minz': 'Robin Minz',
    'R Parag': 'Riyan Parag',
    'R R Pant': 'Rishabh Pant',
    'R Ravindra': 'Rachin Ravindra',
    'R S Kishore': 'Ravisrinivasan Sai Kishore',
    'R Tewatia': 'Rahul Tewatia',
    'Ramandeep Singh': 'Ramandeep Singh',
    'Rashid Khan': 'Rashid-Khan',
    'Rasikh Salam': 'Rasikh Salam',
    'Ravi Bishnoi': 'Ravi Bishnoi',
    'Rinku Singh': 'Rinku Singh',
    'R D Chahar' : 'Rahul Chahar',
    'Rahmanullah Gurbaz' : 'Rahmanullah Gurbaz',
    'R Hangargekar' : 'Rajvardhan Hangargekar',
    'R J W Topley' : 'Reece Topley',
    'R Shepherd' : 'Romario Shepherd',
    'R Powell' : 'Rovman Powell',
    'R Dhawan' : 'Rishi Dhawan',
    'S A Yadav': 'Suryakumar Yadav',
    'S Dube': 'Shivam Dube',
    'S E Rutherford': 'Sherfane Rutherford',
    'S H Johnson': 'Spencer Johnson',
    'S M Curran': 'Sam Curran',
    'S N Thakur': 'Shardul Thakur',
    'S O Hetmeyer': 'Shimron Hetmyer',
    'S P Narine': 'Sunil Narine',
    'S Rizvi': 'Sameer Rizvi',
    'S S Iyer': 'Shreyas Iyer',
    'S Sharma': 'Suyash Sharma',
    'S Shedge': 'Suryansh Shedge',
    'S V Samson': 'Sanju Samson',
    'Sai Sudharsan': 'Sai Sudharsan',
    'Sandeep Sharma': 'Sandeep Sharma',
    'Shahbaz Ahmed': 'Shahbaz Ahmed',
    'Shahrukh Khan': 'Shahrukh Khan',
    'Shubham Dubey': 'Shubham Dubey',
    'Shubman Gill': 'Shubman Gill',
    'Shaik Rasheed': 'Shaik Rasheed',
    'S Singh': 'Simarjeet- Singh',
    'S Singh': 'Shashank Singh',
    'Sachin Baby' : 'Sachin Baby',
    'S Gopal' : 'Shreyas Gopal',
    'Swapnil Singh' : 'Swapnil Singh',
    'S Ravichandran' : 'Smaran Ravichandran',
    'Shrijith Krishnan' : 'KL Shrijith',
    'Shariz Ahmad' : 'Shariz Ahmad',
    'Swastik Chikara' : 'Swastik Chikara',
    'T A Boult': 'Trent Boult',
    'T H David': 'Tim David',
    'T M Head': 'Travis Head',
    'T Stubbs': 'Tristan Stubbs',
    'T U Deshpande': 'Tushar Deshpande',
    'Tilak Varma': 'Tilak Varma',
    'T Natarajan' : 'T Natarajan',
    'Tripurana Vijay' : 'Tripurana Vijay',
    'V G Arora': 'Vaibhav Arora',
    'V Kohli': 'Virat Kohli',
    'V Nigam': 'Vipraj Nigam',
    'V Puthur': 'Vignesh Puthur',
    'V R Iyer': 'Venkatesh Iyer',
    'V Shankar': 'Vijay Shankar',
    'Vijaykumar Vyshak': 'Vyshak Vijaykumar',
    'V Suryavanshi' : 'Vaibhav Suryavanshi',
    'Vishnu Vinod' : 'Vishnu Vinod',
    'Vansh Bedi' : 'Vansh Bedi',
    'W G Jacks': 'Will Jacks',
    'Washington Sundar': 'Washington Sundar',
    'X C Bartlett': 'Xavier Bartlett',
    'Y B K Jaiswal': 'Yashasvi Jaiswal',
    'Y Dayal': 'Yash Dayal',
    'Y S Chahal': 'Yuzvendra Chahal',
    'Y Thakur': 'Yash Thakur',
    'Yudhvir Singh': 'Yudhvir Singh Charak',
    'Yuvraj Chaudhary' : 'Yuvraj Chaudhary',
    'Z Ansari': 'Zeeshan Ansari',
    'A Verma': 'Aniket Verma',
    'R Ghosh': 'Ramakrishna Ghosh',
    'PVD Chameera': 'Dushmantha Chameera',
    'GD Phillips': 'Glenn Phillips',
    'S Joseph': 'Shamar Joseph',
    'A Juyal': 'Aryan Juyal',
    'Mohsin Khan': 'Mohsin Khan',
  }



  # Team-specific manual disambiguation for players with same names

  def disambiguate_duplicates(name, team):
      team = team.upper()

      if name == 'S Singh':
          if team == 'PBKS':
              return 'Shashank Singh'
          elif team == 'SRH':
              return 'Simarjeet- Singh'

      # Add more if needed in future
      return name  # default return original name

  # Apply disambiguation first
  merged['Disambiguated_Player_Name'] = merged.apply(lambda row: disambiguate_duplicates(row['Player Name'], row['Team']), axis=1)

  # Reverse the manual mapping dictionary
  manual_recent_form_mapping_rev = {v.upper(): k.upper() for k, v in manual_recent_form_mapping.items()}

  # Apply manual mapping using disambiguated name
  merged['Mapped_Player_Name'] = merged['Disambiguated_Player_Name'].apply(lambda name: manual_recent_form_mapping_rev.get(name, name))

  # Fuzzy match only where required ===
  def get_best_match_with_score(name, choices, scorer=fuzz.token_sort_ratio, threshold=90):
      if name in choices:
          return name, 100
      match_data = process.extractOne(name, choices, scorer=scorer)
      if match_data:
          match, score, _ = match_data
          if score >= threshold:
              return match, score
      return None, 0

  bat_choices = recent_form_bat['Player'].tolist()
  bowl_choices = recent_form_bowl['Player'].tolist()

  # Batting fuzzy match (if not found in manual mapping)
  def get_bat_match(name):
      if name in bat_choices:
          return name
      return get_best_match_with_score(name, bat_choices)[0]

  # Bowling fuzzy match (if not found in manual mapping)
  def get_bowl_match(name):
      if name in bowl_choices:
          return name
      return get_best_match_with_score(name, bowl_choices)[0]

  merged['bat_match'] = merged['Mapped_Player_Name'].apply(get_bat_match)
  merged['bowl_match'] = merged['Mapped_Player_Name'].apply(get_bowl_match)

  # === Merge Batting Recent Form ===
  merged = pd.merge(
      merged,
      recent_form_bat[['Player', 'recent_form_batting_score']].rename(columns={'Player': 'bat_match'}),
      on='bat_match',
      how='left'
  )

  # === Merge Bowling Recent Form ===
  merged = pd.merge(
      merged,
      recent_form_bowl[['Player', 'recent_form_bowling_score']].rename(columns={'Player': 'bowl_match'}),
      on='bowl_match',
      how='left'
  )

  # === Final Cleanup ===
  merged.drop(['bat_match', 'bowl_match', 'Player_Name'], axis=1, errors='ignore', inplace=True)

  #Effective form score
  # Determine role-based form score
  merged['Effective_Form_Score'] = np.select(
      [
          merged['Player Type'] == 'BAT',
          merged['Player Type'] == 'BOWL',
          merged['Player Type'] == 'ALL'
      ],
      [
          merged['recent_form_batting_score'],
          merged['recent_form_bowling_score'],
          # For allrounder, average of both (can be weighted if needed)
          (merged['recent_form_batting_score'].fillna(0) + merged['recent_form_bowling_score'].fillna(0)) / 2
      ],
      default=0
  )

  # handling class imbalance problem by assigning more weight to new players' recent form
  # Boost score if it's a new player
  merged['Effective_Form_Score'] = np.where(
      merged['is_new_player'] == 1,
      merged['Effective_Form_Score'] * 1.5,
      merged['Effective_Form_Score']
  )

  # Venue and weather features
  match_details = get_match_details_from_last_sheet("SquadPlayerNames_IndianT20League.xlsx", "IPL_2025_Schedule_Final.csv")
  # Safe defaults
  venue_batting_index = 0.5
  venue_bowling_index = 0.5
  venue_rpo = 0.5
  venue_rpw = 0.5

  if match_details:
      venue_batting_index = match_details.get('batting_friendly_index', 0.5)
      venue_bowling_index = match_details.get('bowling_friendly_index', 0.5)
      venue_rpo = match_details.get('rpo_zscore', 0.5)
      venue_rpw = match_details.get('rpw_zscore', 0.5)


  # === Weather ===
  weather_score = 0.5  # Default neutral weather impact
  try:
    weather = fetch_weather(match_details['city']) if match_details else None
    if weather:
        weather_score = calculate_weather_impact(weather)
  except:
    weather = None  # Fail-safe
    pass

    # Toss

  # === Toss ===
  try:
    if toss_decision == 'bat':
      merged['toss_adjustment'] = np.where(
          merged['Team'] == toss_winner,
          0.05 * venue_rpo,
          0
      )
    elif toss_decision == 'field' and weather:
      merged['toss_adjustment'] = np.where(
          merged['Team'] == toss_winner,
          0.05 * (weather['dew_point'] + venue_bowling_index),
          0
      )
    else:
      merged['toss_adjustment'] = 0
  except:
    merged['toss_adjustment'] = 0

  # Calculate venue index based on player_type
  # === Venue Index Mapping based on player type ===
  merged['venue_index'] = np.where(
      merged['Player Type'] == 'BAT',
      venue_batting_index,
      np.where(
          merged['Player Type'] == 'BOWL',
          venue_bowling_index,
          0.6 * venue_batting_index + 0.4 * venue_bowling_index  # For ALL-ROUNDERS
      )
  )

  # Calculate role-specific RPO and RPW based score
  merged['venue_condition_score'] = np.where(
      merged['Player Type'] == 'BAT',
      0.10 * venue_rpo,
      np.where(
          merged['Player Type'] == 'BOWL',
          0.10 * (1 / (venue_rpw + 1e-3)),  # More value to low RPW
          0.05 * venue_rpo + 0.05 * (1 / (venue_rpw + 1e-3))  # Balanced for all-rounders
      )
  )

  # --------------------------------------
  # USING MACHINE LEARNING MODEL HERE
  # --------------------------------------

  # Defining the same features used during training
  ml_features = ['FP_Batting', 'FP_Bowling', 'FP_Fielding','FPbatting','FPbowling']

  # Fill missing values for those columns
  for col in ml_features:
    if col not in merged.columns:
      merged[col] = 0
    merged[col] = merged[col].fillna(0)

  # Predict Final_FP using ML model
  merged['Final FP'] = model.predict(merged[ml_features])

  # Interaction (optional - placeholder)
  merged['interaction_score'] = merged.apply(lambda row: compute_interaction_score(row, merged, player_vs_player_df), axis=1)

  merged['prediction_score'] = (
     0.30 * merged['Effective_Form_Score'] +
     0.20 * merged['Final FP'] +
     0.10 * merged['venue_index'] +
     0.10 * merged['venue_condition_score'] +
     0.05 * (1 - weather_score) +
     0.05 * merged['interaction_score'] +
     merged['toss_adjustment']
   )

  #merged['prediction_score'] = (
  #    0.30 * merged['Effective_Form_Score'] +
  #    0.20 * merged['Final FP'] +
  #    0.05 * merged['venue_index'] +
  #    0.05 * venue_rpo +
  #    0.05 * (1 / (venue_rpw + 1e-3)) +  # avoid division by zero
  #    0.10 * (1 - weather_score) +
  #    0.05 * merged['interaction_score'] +
  #    merged['toss_adjustment']
  #  )

  # Keep only playing or substitute players
  merged = merged[merged['IsPlaying'] != 'NOT_PLAYING']

  # Remove duplicates based on Player Name (keep highest prediction score)
  merged = merged.sort_values(by='prediction_score', ascending=False)

  # Drop duplicate players (keep the one with the highest score)
  merged = merged.drop_duplicates(subset='Player Name', keep='first')
  # Initialize selected team list
  selected_players = []
  total_credits = 0
  max_credits = 100
  team_counts = {}
  type_counts = {'WK': 0, 'BAT': 0, 'ALL': 0, 'BOWL': 0}
  substitute_counter = {}
  total_substitutes = 0

  for _, row in merged.iterrows():
      player_team = row['Team']
      player_type = row['Player Type'].upper()
      player_credit = row['Credits']
      player_status = row['IsPlaying']

      # Credit constraint
      if total_credits + player_credit > max_credits:
          continue

      # Max 10 players from one team
      if team_counts.get(player_team, 0) >= 10:
          continue

      # Max 2 substitutes (only 1 from each team)
      if player_status == 'X_SUBSTITUTE_FACTOR':
          if substitute_counter.get(player_team, 0) >= 1:
              continue
          if total_substitutes >= 2:
              continue

      # Ensure at least 1 WK, 1 ALL, 1 BAT, 1 BOWL in final 11
      if len(selected_players) == 10:
          if type_counts['WK'] == 0 and player_type != 'WK':
              continue  # Reserve slot for WK
          if type_counts['ALL'] == 0 and player_type != 'ALL':
              continue  # Reserve slot for ALL
          if type_counts['BAT'] == 0 and player_type != 'BAT':
              continue  # Reserve slot for BAT
          if type_counts['BOWL'] == 0 and player_type != 'BOWL':
              continue  # Reserve slot for BOWL

      # Select player
      selected_players.append(row)
      total_credits += player_credit
      team_counts[player_team] = team_counts.get(player_team, 0) + 1
      type_counts[player_type] = type_counts.get(player_type, 0) + 1

      if player_status == 'X_SUBSTITUTE_FACTOR':
          substitute_counter[player_team] = substitute_counter.get(player_team, 0) + 1
          total_substitutes += 1

      if len(selected_players) == 11:
          break
  # Final team DataFrame
  team = pd.DataFrame(selected_players)

  # -----------------------------------------
  # Assign Captain and Vice Captain using captain_score
  # -----------------------------------------

  # Compute captain score for each selected player
  team['captain_score'] = team.apply(compute_captain_score, axis=1)

  # Fallback check: all captain scores are NaN or same or zero
  unique_scores = team['captain_score'].dropna().unique()
  fallback_to_prediction = (
      len(unique_scores) <= 1 or  # All same
      team['captain_score'].dropna().sum() == 0  # All zero
  )

  # Assign Roles column first
  team['Role'] = 'N/A'

  if fallback_to_prediction:
    # Fallback to prediction score
    sorted_team = team.sort_values(by='prediction_score', ascending=False).reset_index(drop=True)
    captain = sorted_team.iloc[0]
    vice_captain = sorted_team.iloc[1]
  else:
    # Use captain score-based selection
    sorted_team = team.sort_values(by='captain_score', ascending=False).reset_index(drop=True)
    captain = sorted_team.iloc[0]
    vice_captain = sorted_team.iloc[1] # Select Vice Captain (try to avoid same team as captain)
    for i in range(2, len(sorted_team)):
        candidate = sorted_team.iloc[i]
        if candidate['Team'] != captain['Team'] and candidate['captain_score'] >= 0.9 * vice_captain['captain_score']:
            vice_captain = candidate
            break



  # Assign roles
  team.loc[team['Player Name'] == captain['Player Name'], 'Role'] = 'Captain'
  team.loc[team['Player Name'] == vice_captain['Player Name'], 'Role'] = 'Vice Captain'

  # Drop helper column
  team.drop(columns=['captain_score'], inplace=True)

  # ---------------------------------------------
  # Add Backup Players (top remaining by prediction_score)
  # ---------------------------------------------
  # Get all available players excluding selected
  remaining_pool = merged[~merged['Player Name'].isin(team['Player Name'])].copy()

  # Sort by prediction_score
  remaining_pool = remaining_pool.sort_values(by='prediction_score', ascending=False)

  # Track backup constraints
  backup_players = []
  backup_team_counts = {}

  for _, row in remaining_pool.iterrows():
    team_name = row['Team']
    if backup_team_counts.get(team_name, 0) < 2:
        backup_players.append(row)
        backup_team_counts[team_name] = backup_team_counts.get(team_name, 0) + 1
    if len(backup_players) == 4:
        break

  # Prepare backup DataFrame
  backup_df = pd.DataFrame(backup_players)
  backup_df['Role'] = 'N/A'

  # Combine everything (Captain + VC + remaining 9 + backups)
  # Sort main team with Captain, VC on top
  captain_df = team[team['Role'] == 'Captain']
  vice_captain_df = team[team['Role'] == 'Vice Captain']
  others_df = team[(team['Role'] == 'N/A') & ~team['Player Name'].isin([captain['Player Name'], vice_captain['Player Name']])]

  # Final combined team
  final_team = pd.concat([captain_df, vice_captain_df, others_df, backup_df], ignore_index=True)

  # Output formatting
  output = final_team[['Player Name', 'Team', 'Role']]
  output.columns = ['Player', 'Team', 'Captain/Vice Captain']

  # Save output
  output.to_csv("Predicted_Fantasy_Team_Final.csv", index=False)


# Loading Input File
input_file = "SquadPlayerNames_IndianT20League.xlsx"
# Load all sheet names
sheet_names = pd.ExcelFile(input_file).sheet_names
# Read the last sheet
input_file_df = pd.read_excel(input_file, sheet_name=sheet_names[-1])


#getting match_details from Input Sheet and Schedule Sheet (match_number,venue,city)
result = get_match_details_from_last_sheet("SquadPlayerNames_IndianT20League.xlsx", "IPL_2025_Schedule_Final.csv")
if result:
    match_number = result['match_number']
    venue_name = result['venue']
    city = result['city']
else:
    match_number, venue_name, city = None, None, None

#getting toss details
apikey = "9035a8fa-6838-4959-960c-16a06b79efd5"
match_id, toss_winner, toss_decision = fetch_toss_info(match_number, apikey)

predict_fantasy_team(
    input_file="SquadPlayerNames_IndianT20League.xlsx",
    venue_name=venue_name,
    toss_decision=toss_decision,
    toss_winner=toss_winner,
    city=city
)


(Reading database ... 126487 files and directories currently installed.)
Preparing to unpack google-chrome.deb ...
Unpacking google-chrome-stable (135.0.7049.95-1) over (135.0.7049.95-1) ...
Setting up google-chrome-stable (135.0.7049.95-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
